In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import Window
import pandas as pd

#### Lendo os dados

In [3]:
read_path = '/home/tatiane/Downloads/exemplo_linkage_cases-brazil-cities-time.csv/tmp/exemplo_linkage_cases-brazil-cities-time.csv'

In [4]:
df = spark.read.csv(read_path, sep = ",", multiLine = True)
#base nao apresenta cabeçalho

In [5]:
df.show()

+---+---+------------+------------+---------+---------+------+------+--------+--------+----+----+------------------+
|_c0|_c1|         _c2|         _c3|      _c4|      _c5|   _c6|   _c7|     _c8|     _c9|_c10|_c11|              _c12|
+---+---+------------+------------+---------+---------+------+------+--------+--------+----+----+------------------+
|  0|  0|São Paulo/SP|São Paulo/SP|São Paulo|São Paulo|355030|355030|20200225|20200225|   9|   9|               1.0|
|  2|  2|São Paulo/SP|São Paulo/SP|São Paulo|São Paulo|355030|355030|20200226|20200226|   9|   9|               1.0|
|  4|  4|São Paulo/SP|São Paulo/SP|São Paulo|São Paulo|355030|355030|20200227|20200227|   9|   9|               1.0|
|  6|  6|São Paulo/SP|São Paulo/SP|São Paulo|São Paulo|355030|355030|20200228|20200228|   9|   9|               1.0|
|  1|  1|       TOTAL|       TOTAL|     null|     null|     0|     0|20200225|20200225|   9|   9|0.7237569060773481|
|  5|  5|       TOTAL|       TOTAL|     null|     null|     0|  

In [ ]:
df.printSchema()

#### Pegando informações do linkage

In [11]:
tot = df.count()
print('O número total de registros na base é de: {}'.format(tot))
print('Quantidade de colunas: {}'.format(len(df.columns)))

O número total de registros na base é de: 2248977
Quantidade de colunas: 13


#### Colocando cabeçalho nas colunas

In [12]:
newColumns = ["id1", "id2", "city1", "city2", "regiao_saude1", "regiao_saude2", "codmun1", "codmun2", "date1", "date2", "epi_week1", "epi_week2", "score"]

for x,y in zip(df.columns,newColumns):
    df=df.withColumnRenamed(x,y)

In [13]:
df.printSchema()

root
 |-- id1: string (nullable = true)
 |-- id2: string (nullable = true)
 |-- city1: string (nullable = true)
 |-- city2: string (nullable = true)
 |-- regiao_saude1: string (nullable = true)
 |-- regiao_saude2: string (nullable = true)
 |-- codmun1: string (nullable = true)
 |-- codmun2: string (nullable = true)
 |-- date1: string (nullable = true)
 |-- date2: string (nullable = true)
 |-- epi_week1: string (nullable = true)
 |-- epi_week2: string (nullable = true)
 |-- score: string (nullable = true)



#### Criando colunas para realizar comparações

In [14]:
new_base = df.withColumn('city', F.when(F.col('city1') == F.col('city2'), 1).otherwise(0))
new_base = df.withColumn('regiao_saude', F.when(F.col('regiao_saude1') == F.col('regiao_saude2'), 1).otherwise(0))
new_base = df.withColumn('date', F.when(F.col('date1') == F.col('date2'), 1).otherwise(0))
new_base = df.withColumn('epi_week', F.when(F.col('epi_week1') == F.col('epi_week2'), 1).otherwise(0))
new_base = df.withColumn('codmun', F.when(F.col('codmun1') == F.col('codmun2'), 1).otherwise(0))

In [15]:
new_base.limit(5).toPandas()

,id1,id2,city1,city2,regiao_saude1,regiao_saude2,codmun1,codmun2,date1,date2,epi_week1,epi_week2,score,city,regiao_saude,date,epi_week,codmun
0,0,0,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200225,20200225,9,9,1.0,1,1,1,1,1
1,2,2,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200226,20200226,9,9,1.0,1,1,1,1,1
2,4,4,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200227,20200227,9,9,1.0,1,1,1,1,1
3,6,6,São Paulo/SP,São Paulo/SP,São Paulo,São Paulo,355030,355030,20200228,20200228,9,9,1.0,1,1,1,1,1
4,1,1,TOTAL,TOTAL,None,None,0,0,20200225,20200225,9,9,0.7237569060773481,1,0,1,1,1
